# Notebook 3: Modelltraining & Evaluation

In diesem Notebook werden die vorbereiteten Features geladen, in Trainings- und Test-Sets aufgeteilt und mindestens zwei Modelle (Lineare Regression & Random Forest) trainiert, evaluiert und verglichen. Abschließend wird das beste Modell ausgewählt und für das Deployment vorbereitet.  

## 1. Setup & Laden der Daten

In diesem Abschnitt werden alle benötigten Bibliotheken importiert und der vorbereitete Feature-Datensatz geladen. Ein kurzer Blick auf `head()` und `info()` stellt sicher, dass die Daten korrekt eingelesen wurden.

In [1]:
# 1.1 Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib

# 1.2 Pfad zur finalen Feature-Matrix
DATA_PATH = "../data/processed/cars_features_ready.csv"

# 1.3 Einlesen des Datensatzes
df = pd.read_csv(DATA_PATH)

# 1.4 Grober Check
print("Shape des DataFrames:", df.shape)
display(df.head())
df.info()


Shape des DataFrames: (482324, 25)


,year,condition,odometer,mmr,sale_year,sale_month,sale_day,sale_weekday,make_segment,body,...,season,mean_price_make_model_season,has_sport,has_limited,has_lx,has_se,has_touring,has_premium,miles_per_year,color_popularity
0,2015,5.0,16639.0,20500.0,2014.0,12.0,16.0,1.0,Kia,SUV,...,Winter,14451.380236,0,0,1,0,0,0,-16639.0,2
1,2015,5.0,9393.0,20800.0,2014.0,12.0,16.0,1.0,Kia,SUV,...,Winter,14451.380236,0,0,1,0,0,0,-9393.0,2
2,2014,45.0,1331.0,31900.0,2015.0,1.0,14.0,2.0,BMW,Sedan,...,Winter,16541.470832,0,0,0,1,0,0,1331.0,4
3,2015,41.0,14282.0,27500.0,2015.0,1.0,28.0,2.0,Other,Sedan,...,Winter,11997.357019,0,0,0,0,0,0,14282.0,2
4,2014,43.0,2641.0,66000.0,2014.0,12.0,18.0,3.0,BMW,Sedan,...,Winter,59238.236842,0,0,0,1,0,0,2641.0,4


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 482324 entries, 0 to 482323
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   year                          482324 non-null  int64  
 1   condition                     472891 non-null  float64
 2   odometer                      482269 non-null  float64
 3   mmr                           482324 non-null  float64
 4   sale_year                     482324 non-null  float64
 5   sale_month                    482324 non-null  float64
 6   sale_day                      482324 non-null  float64
 7   sale_weekday                  482324 non-null  float64
 8   make_segment                  482324 non-null  object 
 9   body                          482324 non-null  object 
 10  transmission                  482324 non-null  object 
 11  color                         482324 non-null  object 
 12  interior                      481747 non-nul